In [9]:
!pip install python-dotenv

In [12]:
from openai import OpenAI
from dotenv import load_dotenv
from glob import glob
import os
import PyPDF2
import sys

# Load environment variables from .env file
load_dotenv()

# Get API key from environment variable
api_key = os.getenv("API_KEY")

# Check for the API KEY in the virtual environment
if api_key == None:
    sys.exit('Get your own API KEY! Save it as "API_KEY" in the virtual environment')
else:
    print("API KEY collected successfully! Congratulations :)")

# Instantiate API client
client = OpenAI(api_key=api_key)

os.makedirs('output',exist_ok=True)

def summarize_pdf(pdf_path, **kwargs):

    num_words = kwargs.get("num_words_per_section", 50) 

    # Open PDF file
    with open(pdf_path, 'rb') as file:

        # Create a PDF reader object
        pdf_reader = PyPDF2.PdfReader(file)

        # Initialize an empty string to store the extracted text
        text = ""

        # Iterate over PDF pages
        for page in pdf_reader.pages:
            # Add text to string
            text += page.extract_text()

    # Create a chat completion task to summarize the extracted text
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content":
             f"{text}"},
            {"role": "user", "content": f"Read the paper, and summarize in {num_words} words each about the following:\n\n1. Background\n2. Challenge\n3. Current status\n4. Method\n5. Result\n6. Conclusion\n7. Outlook\n"}
        ]
    )

    # Open a file in write mode
    with open(os.path.join("output","f{pdf_path}_output.txt"), "w") as file:
        # Write output to file
        file.write(completion.choices[0].message.content)

    print(f"Written to {pdf_path}_output.txt")

# # summarize pdf
# for in glob('pdf/*.pdf'):
#     pdf_path = os.path.join("LIFT_paper.pdf")
#     summarize_pdf(pdf_path)

None


In [ ]:
""